In [3]:
import pandas as pd

Series: a one-dimensional labeled array holding data of any type

In [7]:
data = [10, 20, 30, 40, 50]
series = pd.Series(data)
series

0    10
1    20
2    30
3    40
4    50
dtype: int64

DataFrame: a two-dimensional data structure that holds data like a two-dimension array or a table with rows and columns.

In [28]:
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40],
        'City': ['New York', 'San Francisco', 'Los Angeles', 'Chicago']}

df = pd.DataFrame(data)

df

,Name,Age,City
0,Alice,25,New York
1,Bob,30,San Francisco
2,Charlie,35,Los Angeles
3,David,40,Chicago


An index in Pandas is a labeled reference or identifier for each row or element in a data structure, enabling efficient data retrieval and manipulation.

Internally, indexes in Pandas are implemented as hash tables or similar data structures that map index labels to the corresponding data locations, allowing for fast data retrieval.





In [32]:
df.index

RangeIndex(start=0, stop=4, step=1)

In [34]:
# Define a custom index
custom_index = pd.Index(['Person1', 'Person2', 'Person3', 'Person4'])
df.set_index(custom_index)

,Name,Age,City
Person1,Alice,25,New York
Person2,Bob,30,San Francisco
Person3,Charlie,35,Los Angeles
Person4,David,40,Chicago


In [36]:
df.set_index(df["Name"])

,Name,Age,City
Name,,,
Alice,Alice,25,New York
Bob,Bob,30,San Francisco
Charlie,Charlie,35,Los Angeles
David,David,40,Chicago


In [25]:
# Creating a Series with a Multi-level Index
data = [10, 20, 30, 40, 50, 60]
index = pd.MultiIndex.from_tuples([('Q1', 'A'),  ('Q1', 'X'), ('Q1', 'Y'), ('Q2', 'A'), ('Q2', 'B'), ('Q1', 'B'),],
                                  names=['Quarter', 'Region'])
profits_by_quarter_and_bu = pd.Series(data, index=index)

# Display the Multi-level Index Series
profits_by_quarter_and_bu


Quarter  Region
Q1       A         10
         X         20
         Y         30
Q2       A         40
         B         50
Q1       B         60
dtype: int64

In [21]:
# Creating a DataFrame with a Multi-level Index
data = {'Sales': [100, 150, 120, 200, 180, 220]}
index = pd.MultiIndex.from_tuples([('Q1', 'A'), ('Q1', 'B'), ('Q2', 'A'), ('Q2', 'B'), ('Q3', 'A'), ('Q3', 'B')],
                                  names=['Quarter', 'Region'])
df = pd.DataFrame(data, index=index)

# Display the Multi-level Index DataFrame
df

Sales
Quarter Region       
Q1      A         100
        B         150
Q2      A         120
        B         200
Q3      A         180
        B         220

By the way this is one way to add a new column to a dataframe

In [22]:
df["Profits"] = profits_by_quarter_and_bu

In [23]:
df

Sales  Profits
Quarter Region                
Q1      A         100     10.0
        B         150     60.0
Q2      A         120     40.0
        B         200     50.0
Q3      A         180      NaN
        B         220      NaN

Here's another one, equivalent to a FULL OUTER JOIN in SQL

In [26]:
df.combine_first(profits_by_quarter_and_bu.to_frame(name="profits"))

Profits  Sales  profits
Quarter Region                         
Q1      A          10.0  100.0     10.0
        B          60.0  150.0     60.0
        X           NaN    NaN     20.0
        Y           NaN    NaN     30.0
Q2      A          40.0  120.0     40.0
        B          50.0  200.0     50.0
Q3      A           NaN  180.0      NaN
        B           NaN  220.0      NaN

### Equivalents for common operations in SQL

In [38]:
# Create an example DataFrame
data = {
    'EmployeeID': [101, 102, 103, 104, 105],
    'FirstName': ['John', 'Alice', 'Bob', 'Eva', 'Mike'],
    'LastName': ['Smith', 'Johnson', 'Williams', 'Brown', 'Davis'],
    'Department': ['HR', 'Finance', 'IT', 'HR', 'Marketing'],
    'Salary': [55000, 60000, 75000, 52000, 62000]
}

df = pd.DataFrame(data)

df

,EmployeeID,FirstName,LastName,Department,Salary
0,101,John,Smith,HR,55000
1,102,Alice,Johnson,Finance,60000
2,103,Bob,Williams,IT,75000
3,104,Eva,Brown,HR,52000
4,105,Mike,Davis,Marketing,62000


```sql 
SELECT FirstName, LastName
FROM Employees
WHERE Department = 'HR' AND Salary >= 55000;
````

In [40]:
selected_data = df[['FirstName', 'LastName']][(df['Department'] == 'HR') & (df['Salary'] >= 55000)]
selected_data


,FirstName,LastName
0,John,Smith


In [44]:
selected_data = df.loc[(df['Department'] == 'HR') & (df['Salary'] >= 55000), ['FirstName', 'LastName']]
selected_data

,FirstName,LastName
0,John,Smith


In [43]:
selected_data = df.query("Department == 'HR' and Salary >= 55000")[['FirstName', 'LastName']]
selected_data

,FirstName,LastName
0,John,Smith


```sql 
SELECT *
FROM Employees
ORDER BY Salary DESC;
```

In [41]:
sorted_data = df.sort_values(by='Salary', ascending=False)
sorted_data


,EmployeeID,FirstName,LastName,Department,Salary
2,103,Bob,Williams,IT,75000
4,105,Mike,Davis,Marketing,62000
1,102,Alice,Johnson,Finance,60000
0,101,John,Smith,HR,55000
3,104,Eva,Brown,HR,52000


```sql 
SELECT Department, COUNT(*) AS EmployeeCount
FROM Employees
GROUP BY Department;
```

In [42]:
aggregated_data = df.groupby('Department')['EmployeeID'].count().reset_index()
aggregated_data.rename(columns={'EmployeeID': 'EmployeeCount'}, inplace=True)
aggregated_data


,Department,EmployeeCount
0,Finance,1
1,HR,2
2,IT,1
3,Marketing,1


```sql
-- Create a 'Bonus' column with an initial value of zero
ALTER TABLE Employees
ADD Bonus INT DEFAULT 0;

-- Update the 'Bonus' column based on conditions
UPDATE Employees
SET Bonus = CASE
    WHEN Salary < 100000 THEN 5000
    WHEN Salary < 100000 AND Department = 'IT' THEN 10000
    ELSE 0
END;
```

In [45]:
# Create a 'Bonus' column with initial values as zero
df['Bonus'] = 0

# Apply conditions and assign bonuses
df.loc[df['Salary'] < 100000, 'Bonus'] = 5000
df.loc[(df['Salary'] < 100000) & (df['Department'] == 'IT'), 'Bonus'] = 10000
